In [1]:
import cosima_cookbook as cc
import xarray as xr
import glob
import cftime
import xesmf as xe

import dask
from dask.distributed import Client

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43409 Dashboard: /proxy/8787/status,Cluster Workers: 4 Cores: 12 Memory: 48.00 GiB


In [3]:
session = cc.database.create_session() #no argument → cosima-cookbook use the default database

We load that data-arrays containing the grids that each ACCESS-OM2 model configuration used. We need these to assign coordinates with the full (`lon`, `lat`) position of each grid point for the model output.

In [4]:
grid1 = xr.open_dataset('grids/ocean_grid_10.nc')
grid1 = grid1.reset_coords({'geolat_t', 'geolon_t', 'geolat_c', 'geolon_c'})
grid1 = grid1.drop({'xt_ocean', 'yt_ocean', 'xu_ocean', 'yu_ocean'})

grid025 = xr.open_dataset('grids/ocean_grid_025.nc')
grid025 = grid025.reset_coords({'geolat_t', 'geolon_t', 'geolat_c', 'geolon_c'})
grid025 = grid025.drop({'xt_ocean', 'yt_ocean', 'xu_ocean', 'yu_ocean'})

grid010 = xr.open_dataset('grids/ocean_grid_01.nc')
grid010 = grid010.rename({'grid_x_C': 'xu_ocean', 'grid_y_C': 'yu_ocean', 'grid_x_T': 'xt_ocean', 'grid_y_T': 'yt_ocean'})

## Load raw data

We load all raw data from ACCESS-OM2 models and satellite altimetry from CMEMS.

In [5]:
# 1ᵒ RYF: temperature and ssh

expt = '1deg_jra55_ryf9091_gadi'
temp_1_RYF_3D = cc.querying.getvar(expt, 'temp', session, ncfile="ocean.nc", frequency='1 monthly', use_cftime=True)
temp_1_RYF_3D = temp_1_RYF_3D.sel(time=slice('2420', None))
temp_1_RYF_3D = temp_1_RYF_3D.assign_coords({'geolat_t': grid1.geolat_t, 'geolon_t': grid1.geolon_t})
temp_1_RYF_3D = temp_1_RYF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_1_RYF_3D = temp_1_RYF_3D.chunk(chunks='auto')

ssh_1_RYF = cc.querying.getvar(expt, 'sea_level', session, frequency='1 monthly', use_cftime=True)
ssh_1_RYF = ssh_1_RYF.sel(time=slice('2420', None))
ssh_1_RYF = ssh_1_RYF.assign_coords({'geolat_t': grid1.geolat_t, 'geolon_t': grid1.geolon_t})
ssh_1_RYF = ssh_1_RYF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_1_RYF = ssh_1_RYF.chunk(chunks='auto')

In [6]:
temp_1_RYF_3D

<xarray.DataArray 'temp' (time: 3120, depth: 50, y: 300, x: 360)>
dask.array<rechunk-merge, shape=(3120, 50, 300, 360), dtype=float32, chunksize=(6, 50, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.5 -278.5 -277.5 -276.5 ... 76.5 77.5 78.5 79.5
  * y          (y) float64 -77.88 -77.63 -77.38 -77.13 ... 88.87 89.32 89.77
  * depth      (depth) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * time       (time) object 2420-01-16 12:00:00 ... 2679-12-16 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          K
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_conservative_temperature
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3120, nv: 2)>\ndas...

In [7]:
ssh_1_RYF

<xarray.DataArray 'sea_level' (time: 3120, y: 300, x: 360)>
dask.array<rechunk-merge, shape=(3120, 300, 360), dtype=float32, chunksize=(260, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.5 -278.5 -277.5 -276.5 ... 76.5 77.5 78.5 79.5
  * y          (y) float64 -77.88 -77.63 -77.38 -77.13 ... 88.87 89.32 89.77
  * time       (time) object 2420-01-16 12:00:00 ... 2679-12-16 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 7080, nv: 2)>\ndas...

In [8]:
# 1ᵒ IAF: temperature and ssh

expt = '1deg_jra55v13_iaf_spinup1_B1'
temp_1_IAF_3D = cc.querying.getvar(expt, 'temp', session, frequency='1 yearly', ncfile='ocean.nc', use_cftime=True)
temp_1_IAF_3D = temp_1_IAF_3D.sel(time=slice('2018', None))
temp_1_IAF_3D = temp_1_IAF_3D.assign_coords({'geolat_t': grid1.geolat_t, 'geolon_t': grid1.geolon_t})
temp_1_IAF_3D = temp_1_IAF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_1_IAF_3D = temp_1_IAF_3D.chunk(chunks='auto')

ssh_1_IAF = cc.querying.getvar(expt, 'sea_level', session, frequency='1 monthly', use_cftime=True)
ssh_1_IAF = ssh_1_IAF.sel(time=slice('2018', None))
ssh_1_IAF = ssh_1_IAF.assign_coords({'geolat_t': grid1.geolat_t, 'geolon_t': grid1.geolon_t})
ssh_1_IAF = ssh_1_IAF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_1_IAF = ssh_1_IAF.chunk(chunks='auto')

In [9]:
temp_1_IAF_3D

<xarray.DataArray 'temp' (time: 240, depth: 50, y: 300, x: 360)>
dask.array<rechunk-merge, shape=(240, 50, 300, 360), dtype=float32, chunksize=(6, 50, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.5 -278.5 -277.5 -276.5 ... 76.5 77.5 78.5 79.5
  * y          (y) float64 -77.88 -77.63 -77.38 -77.13 ... 88.87 89.32 89.77
  * depth      (depth) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * time       (time) object 2018-07-02 12:00:00 ... 2257-07-02 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          deg_C
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 300, nv: 2)>\ndask...

In [10]:
ssh_1_IAF

<xarray.DataArray 'sea_level' (time: 2880, y: 300, x: 360)>
dask.array<rechunk-merge, shape=(2880, 300, 360), dtype=float32, chunksize=(288, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.5 -278.5 -277.5 -276.5 ... 76.5 77.5 78.5 79.5
  * y          (y) float64 -77.88 -77.63 -77.38 -77.13 ... 88.87 89.32 89.77
  * time       (time) object 2018-01-16 12:00:00 ... 2257-12-16 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(300, 360), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3600, nv: 2)>\ndas...

In [11]:
# 0.25ᵒ RYF: temperature and ssh

expt = '025deg_jra55_ryf9091_gadi'
temp_025_RYF_3D = cc.querying.getvar(expt, variable="temp", session=session, ncfile="ocean.nc", start_time='2300-01-01', use_cftime=True)
temp_025_RYF_3D = temp_025_RYF_3D.sel(time=slice('2300', None))
temp_025_RYF_3D = temp_025_RYF_3D.assign_coords({'geolat_t': grid025.geolat_t, 'geolon_t': grid025.geolon_t})
temp_025_RYF_3D = temp_025_RYF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_025_RYF_3D = temp_025_RYF_3D.chunk(chunks='auto')

ssh_025_RYF = cc.querying.getvar(expt, variable='sea_level', session=session, frequency='1 monthly', start_time='2300-01-01', use_cftime=True)
ssh_025_RYF = ssh_025_RYF.sel(time=slice('2300', None))
ssh_025_RYF = ssh_025_RYF.assign_coords({'geolat_t': grid025.geolat_t, 'geolon_t': grid025.geolon_t})
ssh_025_RYF = ssh_025_RYF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_025_RYF = ssh_025_RYF.chunk(chunks='auto')

In [12]:
temp_025_RYF_3D

<xarray.DataArray 'temp' (time: 3000, depth: 50, y: 1080, x: 1440)>
dask.array<rechunk-merge, shape=(3000, 50, 1080, 1440), dtype=float32, chunksize=(2, 20, 432, 576), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.6 -279.4 -279.1 ... 79.38 79.62 79.88
  * y          (y) float64 -81.08 -80.97 -80.87 -80.76 ... 89.74 89.84 89.95
  * depth      (depth) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * time       (time) object 2300-01-16 12:00:00 ... 2549-12-16 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          K
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_conservative_temperature
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3002, nv: 2)>\ndas...

In [13]:
ssh_025_RYF

<xarray.DataArray 'sea_level' (time: 3000, y: 1080, x: 1440)>
dask.array<rechunk-merge, shape=(3000, 1080, 1440), dtype=float32, chunksize=(20, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.6 -279.4 -279.1 ... 79.38 79.62 79.88
  * y          (y) float64 -81.08 -80.97 -80.87 -80.76 ... 89.74 89.84 89.95
  * time       (time) object 2300-01-16 12:00:00 ... 2549-12-16 12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3024, nv: 2)>\ndas...

In [14]:
# 0.25ᵒ IAF: temperature and ssh

expt = '025deg_jra55v13_iaf_gmredi6'
temp_025_IAF_3D = cc.querying.getvar(expt, "temp", session, ncfile="ocean.nc")
temp_025_IAF_3D = temp_025_IAF_3D.sel(time=slice('2018', None))
temp_025_IAF_3D = temp_025_IAF_3D.assign_coords({'geolat_t': grid025.geolat_t, 'geolon_t': grid025.geolon_t})
temp_025_IAF_3D = temp_025_IAF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_025_IAF_3D = temp_025_IAF_3D.chunk(chunks='auto')

ssh_025_IAF = cc.querying.getvar(expt, variable='sea_level', session=session, frequency='1 monthly')
ssh_025_IAF = ssh_025_IAF.sel(time=slice('2018', None))
ssh_025_IAF = ssh_025_IAF.assign_coords({'geolat_t': grid025.geolat_t, 'geolon_t': grid025.geolon_t})
ssh_025_IAF = ssh_025_IAF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_025_IAF = ssh_025_IAF.chunk(chunks='auto')

In [15]:
temp_025_IAF_3D

<xarray.DataArray 'temp' (time: 240, depth: 50, y: 1080, x: 1440)>
dask.array<rechunk-merge, shape=(240, 50, 1080, 1440), dtype=float32, chunksize=(2, 20, 432, 576), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.6 -279.4 -279.1 ... 79.38 79.62 79.88
  * y          (y) float64 -81.08 -80.97 -80.87 -80.76 ... 89.74 89.84 89.95
  * depth      (depth) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * time       (time) datetime64[ns] 2018-06-30T12:00:00 ... 2257-06-30T12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          deg_C
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 300, nv: 2)>\ndask...

In [16]:
ssh_025_IAF

<xarray.DataArray 'sea_level' (time: 2880, y: 1080, x: 1440)>
dask.array<rechunk-merge, shape=(2880, 1080, 1440), dtype=float32, chunksize=(20, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.6 -279.4 -279.1 ... 79.38 79.62 79.88
  * y          (y) float64 -81.08 -80.97 -80.87 -80.76 ... 89.74 89.84 89.95
  * time       (time) datetime64[ns] 2018-01-14T12:00:00 ... 2257-12-14T12:00:00
    latitude   (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 3600, nv: 2)>\ndas...

In [17]:
# 0.10ᵒ RYF: temperature and ssh

expt = '01deg_jra55v13_ryf9091'
temp_010_RYF_3D = cc.querying.getvar(expt, 'temp', session, ncfile="ocean.nc", start_time='1950-01-01', use_cftime=True)
temp_010_RYF_3D = temp_010_RYF_3D.sel(time=slice('1950', None))
temp_010_RYF_3D = temp_010_RYF_3D.assign_coords({'geolat_t': grid010.geolat_t, 'geolon_t': grid010.geolon_t})
temp_010_RYF_3D = temp_010_RYF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_010_RYF_3D = temp_010_RYF_3D.chunk(chunks='auto');

ssh_010_RYF = cc.querying.getvar(expt, variable='sea_level', session=session, frequency='1 monthly', start_time='1950-01-01', use_cftime=True)
ssh_010_RYF = ssh_010_RYF.sel(time=slice('1950', None))
ssh_010_RYF = ssh_010_RYF.assign_coords({'geolat_t': grid010.geolat_t, 'geolon_t': grid010.geolon_t})
ssh_010_RYF = ssh_010_RYF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_010_RYF = ssh_010_RYF.chunk(chunks='auto')

In [18]:
temp_010_RYF_3D

<xarray.DataArray 'temp' (time: 2754, depth: 75, y: 2700, x: 3600)>
dask.array<rechunk-merge, shape=(2754, 75, 2700, 3600), dtype=float32, chunksize=(2, 14, 600, 800), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * y          (y) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * depth      (depth) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time       (time) object 1950-01-16 12:00:00 ... 2179-12-16 12:00:00
    latitude   (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          deg_C
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 2755, nv: 2)>\ndas...

In [19]:
ssh_010_RYF

<xarray.DataArray 'sea_level' (time: 2754, y: 2700, x: 3600)>
dask.array<rechunk-merge, shape=(2754, 2700, 3600), dtype=float32, chunksize=(3, 2025, 2700), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * y          (y) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * time       (time) object 1950-01-16 12:00:00 ... 2179-12-16 12:00:00
    latitude   (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 2757, nv: 2)>\ndas...

In [20]:
# 0.10ᵒ IAF: temperature

temp_010_cycle1_IAF_3D = cc.querying.getvar('01deg_jra55v140_iaf', 'temp', session, ncfile='ocean-3d-temp-1-monthly-mean-ym_%.nc',
                          frequency='1 monthly',
                          start_time='1958-01-31 00:00:00', 
                          end_time='2018-12-31 00:00:00',
                          use_cftime=True)

temp_010_cycle2_IAF_3D = cc.querying.getvar('01deg_jra55v140_iaf_cycle2', 'temp', session, ncfile='ocean-3d-temp-1-monthly-mean-ym_%.nc',
                          frequency='1 monthly',
                          start_time='1958-01-31 00:00:00', 
                          end_time='2018-12-31 00:00:00',
                          use_cftime=True)

howmanymonths = str(temp_010_cycle1_IAF_3D.time.size + 1) + 'M'
temp_010_cycle2_IAF_3D = temp_010_cycle2_IAF_3D.assign_coords(time = temp_010_cycle2_IAF_3D.time + xr.coding.cftime_offsets.to_offset(howmanymonths))

temp_010_cycles1_and_2_IAF_3D = xr.concat([temp_010_cycle1_IAF_3D, temp_010_cycle2_IAF_3D], 'time')

temp_010_cycle3_IAF_3D = cc.querying.getvar('01deg_jra55v140_iaf_cycle3', 'temp', session, ncfile='ocean-3d-temp-1-monthly-mean-ym_%.nc',
                          frequency='1 monthly',
                          start_time='1958-01-31 00:00:00', 
                          end_time='2018-12-31 00:00:00',
                          use_cftime=True)

howmanymonths = str(temp_010_cycles1_and_2_IAF_3D.time.size + 1) + 'M'
temp_010_cycle3_IAF_3D = temp_010_cycle3_IAF_3D.assign_coords(time = temp_010_cycle3_IAF_3D.time + xr.coding.cftime_offsets.to_offset(howmanymonths))

temp_010_IAF_3D = xr.concat([temp_010_cycles1_and_2_IAF_3D, temp_010_cycle3_IAF_3D], 'time')

temp_010_IAF_3D = temp_010_IAF_3D.assign_coords({'geolat_t': grid010.geolat_t, 'geolon_t': grid010.geolon_t})
temp_010_IAF_3D = temp_010_IAF_3D.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'st_ocean': 'depth', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
temp_010_IAF_3D = temp_010_IAF_3D.chunk(chunks='auto');

In [21]:
temp_010_IAF_3D

<xarray.DataArray 'temp' (time: 2196, depth: 75, y: 2700, x: 3600)>
dask.array<rechunk-merge, shape=(2196, 75, 2700, 3600), dtype=float32, chunksize=(2, 38, 270, 360), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * y          (y) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * depth      (depth) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time       (time) object 1958-01-16 12:00:00 ... 2140-12-31 12:00:00
    latitude   (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Attributes:
    long_name:      Conservative temperature
    units:          K
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_water_conservative_temperature
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 732, nv: 2)>\ndask...

In [22]:
# 0.10ᵒ IAF: ssh

ssh_010_cycle1_IAF = cc.querying.getvar('01deg_jra55v140_iaf', variable='sea_level', session=session,
                                        ncfile='ocean-2d-sea_level-1-monthly-mean-ym_%.nc',
                                        start_time='1958-01-1 00:00:00',
                                        end_time='2018-12-31 00:00:00',
                                        use_cftime=True)

ssh_010_cycle2_IAF = cc.querying.getvar('01deg_jra55v140_iaf_cycle2', variable='sea_level', session=session,
                                        ncfile='ocean-2d-sea_level-1-monthly-mean-ym_%.nc',
                                        use_cftime=True)

howmanymonths = str(ssh_010_cycle1_IAF.time.size + 1) + 'M'
ssh_010_cycle2_IAF = ssh_010_cycle2_IAF.assign_coords(time = ssh_010_cycle2_IAF.time + xr.coding.cftime_offsets.to_offset(howmanymonths))

ssh_010_IAF_cycles1_and_2 = xr.concat([ssh_010_cycle1_IAF, ssh_010_cycle2_IAF], 'time')

ssh_010_cycle3_IAF = cc.querying.getvar('01deg_jra55v140_iaf_cycle3', variable='sea_level', session=session,
                                        ncfile='ocean-2d-sea_level-1-monthly-mean-ym_%.nc',
                                        use_cftime=True)

howmanymonths = str(ssh_010_IAF_cycles1_and_2.time.size + 1) + 'M'
ssh_010_cycle3_IAF = ssh_010_cycle3_IAF.assign_coords(time = ssh_010_cycle3_IAF.time + xr.coding.cftime_offsets.to_offset(howmanymonths))

ssh_010_IAF = xr.concat([ssh_010_IAF_cycles1_and_2, ssh_010_cycle3_IAF], 'time')

ssh_010_IAF = ssh_010_IAF.assign_coords({'geolat_t': grid010.geolat_t, 'geolon_t': grid010.geolon_t})
ssh_010_IAF = ssh_010_IAF.rename({'xt_ocean': 'x', 'yt_ocean': 'y', 'geolon_t': 'longitude', 'geolat_t': 'latitude'})
ssh_010_IAF = ssh_010_IAF.chunk(chunks='auto')

In [23]:
ssh_010_IAF

<xarray.DataArray 'sea_level' (time: 2196, y: 2700, x: 3600)>
dask.array<rechunk-merge, shape=(2196, 2700, 3600), dtype=float32, chunksize=(4, 2160, 2880), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * y          (y) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * time       (time) object 1958-01-16 12:00:00 ... 2140-12-31 12:00:00
    latitude   (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Attributes:
    long_name:      effective sea level (eta_t + patm/(rho0*g)) on T cells
    units:          meter
    valid_range:    [-1000.  1000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_height_above_geoid
    time_bounds:    <xarray.DataArray 'time_bounds' (time: 732, nv: 2)>\ndask...

In [24]:
# ssh CMEMS (observations, satellite altimetry)

# load the daily gridded CMEMS; this product is at /g/data/ua8/CMEMS_SeaLevel in Australia's HPC @ NCI
filenames = glob.glob("/g/data/ua8/CMEMS_SeaLevel/v2019/*/*.nc")
cmems = xr.open_mfdataset(filenames, parallel=True)

ssh_cmems = cmems.adt
ssh_cmems = ssh_cmems.sel(time=slice('1993-01', '2019-12'))
ssh_cmems = ssh_cmems.chunk(chunks='auto')
ssh_cmems = ssh_cmems.rename('ssh_cmems')
ssh_cmems = ssh_cmems.resample(time='1MS').mean()

In [25]:
ssh_cmems

<xarray.DataArray 'ssh_cmems' (time: 324, latitude: 720, longitude: 1440)>
dask.array<stack, shape=(324, 720, 1440), dtype=float64, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2019-12-01
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9

## Regrid using `xesfm`

We regrid all output onto a regular longitude-latitude grid with 1ᵒ lateral resolution.

First we create a placeholder dataset `ds_out` that we will to construct the regridding weights via `xesfm` package. We use a longitude range `lon` $\in (-280, 80)$ to match the longitudes the model input uses but, *most importantly* to plot global fields without a "seam" in the middle of the Pacific Ocean.

In [28]:
ds_out = xe.util.grid_global(1, 1)
ds_out = ds_out.drop({'lon_b', 'lat_b'})
ds_out = ds_out.assign_coords({'lon': ds_out.lon - 100.})
ds_out = ds_out.rename({'lon': 'longitude', 'lat': 'latitude'})
ds_out  # contains lat/lon values of cell centers and boundaries.

<xarray.Dataset>
Dimensions:    (x: 360, y: 180)
Coordinates:
    longitude  (y, x) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
    latitude   (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimensions without coordinates: x, y
Data variables:
    *empty*

### Construct the regridders, regrid, and save output

Let's begin with the CMEMS data.

In [32]:
regridder_025degcmems_1deg = xe.Regridder(ssh_cmems, ds_out, 'bilinear', periodic=True,
                                          filename="interpolation_weights/bilinear_tracer_weights_in025degcmems_out1deg.nc",
                                          reuse_weights=True)

regridder_025degcmems_1deg

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            interpolation_weights/bilinear_tracer_weights_in025degcmems_out1deg.nc 
Reuse pre-computed weights? True 
Input grid shape:           (720, 1440) 
Output grid shape:          (180, 360) 
Periodic in longitude?      True

In [31]:
ssh_cmems_regridded = regridder_025degcmems_1deg(ssh_cmems)
ssh_cmems_regridded = ssh_cmems_regridded.rename('ssh')
ssh_cmems_regridded = ssh_cmems_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_cmems_regridded = ssh_cmems_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_cmems_regridded

<xarray.DataArray 'ssh' (time: 324, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(324, 180, 360), dtype=float64, chunksize=(1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2019-12-01
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [29]:
output_folder = "output/cmems-monthlymean-regridded/"

years, das = zip(*ssh_cmems_regridded.groupby("time.year"))
ds = [d.to_dataset(name="ssh") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

And now proceed with the ACCESS-OM2 data.

In [34]:
regridder_1degACCESSOM2_1deg = xe.Regridder(temp_1_RYF_3D.drop({'x', 'y'}), ds_out, 'bilinear', periodic=True,
                                            filename="interpolation_weights/bilinear_tracer_weights_in1degACCESSOM2_out1deg.nc",
                                            reuse_weights=True)

regridder_1degACCESSOM2_1deg

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            interpolation_weights/bilinear_tracer_weights_in1degACCESSOM2_out1deg.nc 
Reuse pre-computed weights? True 
Input grid shape:           (300, 360) 
Output grid shape:          (180, 360) 
Periodic in longitude?      True

In [35]:
regridder_025degACCESSOM2_1deg = xe.Regridder(temp_025_RYF_3D.drop({'x', 'y'}), ds_out, 'bilinear', periodic=True,
                                              filename="interpolation_weights/bilinear_tracer_weights_in025degACCESSOM2_out1deg.nc",
                                              reuse_weights=True)

regridder_025degACCESSOM2_1deg

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            interpolation_weights/bilinear_tracer_weights_in025degACCESSOM2_out1deg.nc 
Reuse pre-computed weights? True 
Input grid shape:           (1080, 1440) 
Output grid shape:          (180, 360) 
Periodic in longitude?      True

In [36]:
regridder_010degACCESSOM2_1deg = xe.Regridder(temp_010_RYF_3D.drop({'x', 'y'}), ds_out, 'bilinear', periodic=True,
                                              filename="interpolation_weights/bilinear_tracer_weights_in010degACCESSOM2_out1deg.nc",
                                              reuse_weights=True)
regridder_010degACCESSOM2_1deg

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            interpolation_weights/bilinear_tracer_weights_in010degACCESSOM2_out1deg.nc 
Reuse pre-computed weights? True 
Input grid shape:           (2700, 3600) 
Output grid shape:          (180, 360) 
Periodic in longitude?      True

In [36]:
ssh_1_IAF_regridded = regridder_1degACCESSOM2_1deg(ssh_1_IAF)
ssh_1_IAF_regridded = ssh_1_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_1_IAF_regridded = ssh_1_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_1_IAF_regridded

<xarray.DataArray 'sea_level' (time: 2880, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(2880, 180, 360), dtype=float64, chunksize=(288, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2018-01-16 12:00:00 ... 2257-12-16 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [38]:
output_folder = "output/ssh-1deg-IAF-regridded/"

years, das = zip(*ssh_1_IAF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [37]:
ssh_1_RYF_regridded = regridder_1degACCESSOM2_1deg(ssh_1_RYF)
ssh_1_RYF_regridded = ssh_1_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_1_RYF_regridded = ssh_1_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_1_RYF_regridded

<xarray.DataArray 'sea_level' (time: 3120, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(3120, 180, 360), dtype=float64, chunksize=(260, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2420-01-16 12:00:00 ... 2679-12-16 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [39]:
output_folder = "output/ssh-1deg-RYF-regridded/"

years, das = zip(*ssh_1_RYF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [40]:
ssh_025_IAF_regridded = regridder_025degACCESSOM2_1deg(ssh_025_IAF.chunk({'time': 48}))
ssh_025_IAF_regridded = ssh_025_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_025_IAF_regridded = ssh_025_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_025_IAF_regridded

<xarray.DataArray 'sea_level' (time: 2880, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(2880, 180, 360), dtype=float64, chunksize=(48, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2018-01-14 12:00:00 ... 2257-12-14 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [43]:
output_folder = "output/ssh-025deg-IAF-regridded/"

years, das = zip(*ssh_025_IAF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [16]:
ssh_025_RYF_regridded = regridder_025degACCESSOM2_1deg(ssh_025_RYF.chunk({'time': 48}))
ssh_025_RYF_regridded = ssh_025_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_025_RYF_regridded = ssh_025_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_025_RYF_regridded

<xarray.DataArray 'sea_level' (time: 3000, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(3000, 180, 360), dtype=float64, chunksize=(48, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2300-01-16 12:00:00 ... 2549-12-16 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [17]:
output_folder = "output/ssh-025deg-RYF-regridded/"

years, das = zip(*ssh_025_RYF_regridded.groupby("time.year"))
years, das = years[-100:], das[-100:]
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [20]:
ssh_010_RYF_regridded = regridder_010degACCESSOM2_1deg(ssh_010_RYF.chunk({'x': None, 'y': None, 'time':2}))
ssh_010_RYF_regridded = ssh_010_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_010_RYF_regridded = ssh_010_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_010_RYF_regridded

<xarray.DataArray 'sea_level' (time: 243, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(243, 180, 360), dtype=float64, chunksize=(2, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 2149-10-16 12:00:00 ... 2169-12-16 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [21]:
output_folder = "output/ssh-010deg-RYF-regridded/"

years, das = zip(*ssh_010_RYF_regridded.groupby("time.year"))
years, das = years[-3:], das[-3:]
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]
xr.save_mfdataset(ds, paths)

In [37]:
ssh_010_IAF_regridded = regridder_010degACCESSOM2_1deg(ssh_010_IAF.chunk({'x': None, 'y': None, 'time':20}))
ssh_010_IAF_regridded = ssh_010_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
ssh_010_IAF_regridded = ssh_010_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

ssh_010_IAF_regridded

<xarray.DataArray 'sea_level' (time: 2196, latitude: 180, longitude: 360)>
dask.array<transpose, shape=(2196, 180, 360), dtype=float64, chunksize=(20, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 1958-01-16 12:00:00 ... 2140-12-31 12:00:00
  * longitude  (longitude) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * latitude   (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear

In [53]:
output_folder = "output/ssh-010deg-IAF-regridded/"

years, das = zip(*ssh_010_IAF_regridded.groupby("time.year"))
years
ds = [d.to_dataset(name="sea_level") for d in das]
paths = [output_folder + "ssh-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [ ]:
temp_1_RYF_regridded = regridder_1degACCESSOM2_1deg(temp_1_RYF_3D.chunk({'x': None, 'y': None, 'time': 1, 'depth': 4}))
temp_1_RYF_regridded = temp_1_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_1_RYF_regridded = temp_1_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

temp_1_RYF_regridded

In [ ]:
output_folder = "output/temp-1deg-RYF-regridded/"

years, das = zip(*temp_1_RYF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [ ]:
temp_1_IAF_regridded = regridder_1degACCESSOM2_1deg(temp_1_IAF_3D.chunk({'x': None, 'y': None, 'time': 1, 'depth': 4}))
temp_1_IAF_regridded = temp_1_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_1_IAF_regridded = temp_1_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})
temp_1_IAF_regridded 

In [ ]:
output_folder = "output/temp-1deg-IAF-regridded/"

years, das = zip(*temp_1_IAF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [18]:
temp_025_RYF_regridded = regridder_025degACCESSOM2_1deg(temp_025_RYF_3D.chunk({'x': None, 'y': None, 'time': 1, 'depth': None}))
temp_025_RYF_regridded = temp_025_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_025_RYF_regridded = temp_025_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

temp_025_RYF_regridded

,Array,Chunk
Bytes,77.76 GB,25.92 MB
Shape,"(3000, 50, 180, 360)","(1, 50, 180, 360)"
Count,1259126 Tasks,3000 Chunks
Type,float64,numpy.ndarray


In [19]:
output_folder = "output/temp-025deg-RYF-regridded/"

years, das = zip(*temp_025_RYF_regridded.groupby("time.year"))
years, das = years[-100:], das[-100:]
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [38]:
temp_025_IAF_regridded = regridder_025degACCESSOM2_1deg(temp_025_IAF_3D.chunk({'x': None, 'y': None, 'time': 1, 'depth': 4}))
temp_025_IAF_regridded = temp_025_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_025_IAF_regridded = temp_025_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})

temp_025_IAF_regridded 

,Array,Chunk
Bytes,5.79 GiB,1.98 MiB
Shape,"(240, 50, 180, 360)","(1, 4, 180, 360)"
Count,129994 Tasks,3120 Chunks
Type,float64,numpy.ndarray


In [ ]:
output_folder = "output/temp-025deg-IAF-regridded/"

years, das = zip(*temp_025_IAF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [24]:
temp_010_RYF_regridded = regridder_010degACCESSOM2_1deg(temp_010_RYF_3D.sel(time=slice('2155', None)).chunk({'x': None, 'y': None, 'time': 1, 'depth': 4}))
temp_010_RYF_regridded = temp_010_RYF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_010_RYF_regridded = temp_010_RYF_regridded.rename({'x': 'longitude', 'y': 'latitude'})
temp_010_RYF_regridded = temp_010_RYF_regridded.chunk({'depth': None})

temp_010_RYF_regridded

,Array,Chunk
Bytes,7.00 GB,38.88 MB
Shape,"(180, 75, 180, 360)","(1, 75, 180, 360)"
Count,536805 Tasks,180 Chunks
Type,float64,numpy.ndarray


In [25]:
output_folder = "output/temp-010deg-RYF-regridded/"

years, das = zip(*temp_010_RYF_regridded.groupby("time.year"))
# years, das = (years[-4], ), (das[-4], )
years, das = years[-4:], das[-4:]
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

In [ ]:
temp_010_IAF_regridded = regridder_010degACCESSOM2_1deg(temp_010_IAF_3D.chunk({'x': None, 'y': None, 'time': 1, 'depth': None}))
temp_010_IAF_regridded = temp_010_IAF_regridded.assign_coords({'x': ds_out.longitude[0, :], 'y': ds_out.latitude[:, 0]})
temp_010_IAF_regridded = temp_010_IAF_regridded.rename({'x': 'longitude', 'y': 'latitude'})
temp_010_IAF_regridded = temp_010_IAF_regridded.chunk({'depth': None})

temp_010_IAF_regridded

In [ ]:
output_folder = "/g/data/v45/nc3020/LowFreqVariability/output-revision/temp-010deg-IAF-regridded/"

years, das = zip(*temp_010_IAF_regridded.groupby("time.year"))
ds = [d.to_dataset(name="temp") for d in das]
paths = [output_folder + "temp-{}.nc".format(y) for y in years]

xr.save_mfdataset(ds, paths)

## Reduce 3D the `temp` fields to 2D Upper-Ocean Heat Content (UOHC)

UOHC is computed by integrating `temp` over some depth and multiply by appropriate constants:

$$
\mathrm{UOHC}(\texttt{lon}, \texttt{lat}) = \rho_0 c_p \int_{\mathrm{bottom}}^{\mathrm{top}} T(\texttt{lon}, \texttt{lat}, z) \, \mathrm{d} z .
$$

In [40]:
temp_1_RYF = xr.open_mfdataset('output/temp-1deg-RYF-regridded/*.nc', combine='by_coords', parallel=True)
temp_1_RYF = temp_1_RYF.temp

temp_1_IAF = xr.open_mfdataset('output/temp-1deg-IAF-regridded/*.nc', combine='by_coords', parallel=True)
temp_1_IAF = temp_1_IAF.temp

temp_025_RYF = xr.open_mfdataset('output/temp-025deg-RYF-regridded/*.nc', combine='by_coords', parallel=True)
temp_025_RYF = temp_025_RYF.temp

temp_025_IAF = xr.open_mfdataset('output/temp-025deg-IAF-regridded/*.nc', combine='by_coords', parallel=True)
temp_025_IAF = temp_025_IAF.temp

temp_010_RYF = xr.open_mfdataset('output/temp-010deg-RYF-regridded/*.nc', combine='by_coords', parallel=True)
temp_010_RYF = temp_010_RYF.temp

temp_010_IAF = xr.open_mfdataset('output/temp-010deg-IAF-regridded/*.nc', combine='by_coords', parallel=True)
temp_010_IAF = temp_010_IAF.temp

In [9]:
def calculate_UOHC(temp, top, bottom, attrib=""):
    # find the actual depth of cells that are closer to `top` and `bottom` values
    Lz = np.max(temp.sel(depth=slice(top, bottom)).depth.values) - np.min(temp.sel(depth=slice(top, bottom)).depth.values)
    
    cp = 3992.1 # J degK**-1 kg**-1
    ρ0 = 1035.0 # kg m**−3

    requested_depth = top - bottom
    grid_depth = (1 + 0*temp.isel(time=0).sel({'longitude': -40, 'latitude': 40}, method='nearest')).sel(depth=slice(top, bottom)).integrate('depth').values
    
    normalization_for_depth = requested_depth / grid_depth
    
    UOHC = (normalization_for_depth * ρ0 * cp * temp).sel(depth=slice(top, bottom)).integrate('depth')
    UOHC = UOHC.rename("uohc")
    UOHC = UOHC.chunk({"time": 12})
    UOHC.attrs["long_name"] = "Upper ocean heat content; depth-integrated from "+str(bottom)+"m to "+str(top)+"m"
    UOHC.attrs["units"] = "Joules meters**-2"
    UOHC.attrs["coordinates"] = "longitude, latitude"
    UOHC.attrs["extra info"] = attrib
    
    return UOHC

We compute the UOHC for the following top/bottom depth combinations (in meters).

In [42]:
top_bottom_depths = [[0, 20], [0, 50], [0, 200], [20, 200], [20 ,50]]

In [ ]:
for top_bottom_depth in top_bottom_depths:
    top, bottom = top_bottom_depth[0], top_bottom_depth[1]

    print('top: ', top)
    print('bottom: ', bottom)
    
    UOHC_1deg = calculate_UOHC(temp_1_RYF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-1 output")
    UOHC_025deg = calculate_UOHC(temp_025_RYF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-025 output")
    UOHC_010deg = calculate_UOHC(temp_010_RYF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-010 output")

    output_folder = "output/uohc-1deg-"+str(top)+"m-"+str(bottom)+"m-RYF/"
    print(output_folder)

    years, das = zip(*UOHC_1deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)

    
    output_folder = "output/uohc-025deg-"+str(top)+"m-"+str(bottom)+"m-RYF/"

    years, das = zip(*UOHC_025deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)


    output_folder = "output/uohc-010deg-"+str(top)+"m-"+str(bottom)+"m-RYF/"

    years, das = zip(*UOHC_010deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)
    
    print(' ')

In [ ]:
for top_bottom_depth in top_bottom_depths:
    top, bottom = top_bottom_depth[0], top_bottom_depth[1]

    print('top: ', top)
    print('bottom: ', bottom)
    
    UOHC_1deg = calculate_UOHC(temp_1_IAF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-1 output")
    UOHC_025deg = calculate_UOHC(temp_025_IAF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-025 output")
    UOHC_010deg = calculate_UOHC(temp_010_IAF, top, bottom, "interpolated to 1deg lat-lon grid from ACCESS-OM2-010 output")

    output_folder = "output/uohc-1deg-"+str(top)+"m-"+str(bottom)+"m-IAF/"
    print(output_folder)

    years, das = zip(*UOHC_1deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)

    
    output_folder = "output/uohc-025deg-"+str(top)+"m-"+str(bottom)+"m-IAF/"

    years, das = zip(*UOHC_025deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)


    output_folder = "output/uohc-010deg-"+str(top)+"m-"+str(bottom)+"m-IAF/"

    years, das = zip(*UOHC_010deg.groupby("time.year"))
    ds = [d.to_dataset(name="uohc") for d in das]
    paths = [output_folder + "uohc-{}.nc".format(y) for y in years]
    xr.save_mfdataset(ds, paths)
    
    print(' ')

#### Packages used in notebook

In [2]:
%load_ext watermark

In [3]:
%watermark --iversions

dask           : 2021.5.0
cftime         : 1.4.1
cosima_cookbook: 0.5.8
xesmf          : 0.5.3
xarray         : 0.18.2

